# <b> Canal de diffusion anonyme et génération de secret </b>

<i> Compte-rendu du TP2 par Valentin SÉRÉE et Jérémy SERENA [INFO3]</i>

In [5]:
from datetime import datetime

class Forum:
    messages = []

    def init(self):
        self.messages = []

    def recupDate(self):
        dateBrute = str(datetime.now())
        date = dateBrute[11:13] + "h" + dateBrute[14:16] + "m" + dateBrute[17:19] + "s"
        return date

    def calculDate(self, date2):
            msg_date = list(date2)
            date = ""
            i=0
            while msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            heure = int(date[0:2])
            date = ""
            i += 1
            while msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            minute = int(date[0:2])
            date = ""
            i += 1
            while len(msg_date) and msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            seconde = int(date[0:2])
            total = 3600*heure+60*minute+seconde
            return (total)
    
    def recupererMessagesAnonymes(self, date_debut, date_fin) :
        for msg in self.messages :
            date = self.calculDate(msg[1])
            date_start = self.calculDate(date_debut)
            date_end = self.calculDate(date_fin)
            if date > date_start and date < date_end :
                print(msg)

    def posterMessageAnonyme(self, message) :
        date = self.recupDate()
        self.messages.append([message, date])



IndentationError: expected an indented block (<ipython-input-5-6b1f04c16188>, line 45)

In [ ]:
if __name__ == "__main__" :
    print("start")
    forum = Forum()
    while(True) :
        string = input()
        if "show_message" == string :
            print("entrez la date au format 11h02m03s" )
            date_debut = input()
            date_fin = input()
            forum.recupererMessagesAnonymes(date_debut, date_fin)
        else :
            forum.posterMessageAnonyme(string)

In [8]:
import time
import random

class User :
    name = ""

    def __init__(self, nom) :
        self.name = nom

    def genererSecret(self, forum, nom1, nom2, duree) :
        while duree > 0 :
            attente = random.randint(0, 2)
            time.sleep(attente)
            bit = random.randint(0, 1)
            if bit == 0 :
                message = nom1
            else :
                message = nom2
            forum.posterMessageAnonyme(message)
            duree -= attente


if __name__ == "__main__" :
    print("start")
    forum = Forum()
    bob = User("bob")
    alice = User("alice")
    time.sleep(1)
    bob.genererSecret(forum, "bob", "alice", 20)
    alice.genererSecret(forum, "alice", "bob", 20)
    print(forum, bob, alice)
    time.sleep(2)
    forum.recupererMessagesAnonymes("08l08l08l", "10h10h10h") 
    

start
<__main__.Forum object at 0x000001664065E550> <__main__.User object at 0x00000166412557C0> <__main__.User object at 0x0000016641255070>
[<__main__.User object at 0x0000016641255490>, '09h18m43s']
[<__main__.User object at 0x0000016641255340>, '09h18m43s']
[<__main__.User object at 0x0000016641255340>, '09h18m43s']
[<__main__.User object at 0x0000016641255490>, '09h18m44s']
[<__main__.User object at 0x0000016641255490>, '09h18m44s']
[<__main__.User object at 0x0000016641255340>, '09h18m45s']
[<__main__.User object at 0x0000016641255490>, '09h18m47s']
[<__main__.User object at 0x0000016641255340>, '09h18m49s']
[<__main__.User object at 0x0000016641255340>, '09h18m51s']
[<__main__.User object at 0x0000016641255340>, '09h18m53s']
[<__main__.User object at 0x0000016641255D60>, '09h19m03s']
[<__main__.User object at 0x0000016641255D60>, '09h19m05s']
[<__main__.User object at 0x0000016641255D60>, '09h19m06s']
[<__main__.User object at 0x0000016641255700>, '09h19m06s']
[<__main__.User ob